### Need to import a few tools/libraries to get started with our task

In [ ]:
import requests
import http
import json
import csv
import configparser
import requests
from urllib3.exceptions import InsecureRequestWarning

config = configparser.ConfigParser()
# Suppress only the single warning from urllib3 needed.
requests.packages.urllib3.disable_warnings(category=InsecureRequestWarning)
print("Import Success")

In [ ]:
config.read("./BackBoxConfig.ini")
BACKBOXIP = config["BACKBOX"]["BACKBOXIP"] #"192.168.xxx.xxx"
BACKBOXUSER = config["BACKBOX"]["BACKBOXUSER"]   #'username'
BACKBOXUSERPASS = config["BACKBOX"]["BACKBOXUSERPASS"]    #'password'

### To begin we need to get all of the needed data to run our script together. 

In [ ]:
#information needed to reach out to backbox API
BackBoxBaseURL = "https://" + BACKBOXIP #example 192.168.1.2
BackBoxUser = BACKBOXUSER #enter your user name for BackBox
BackBoxPassword = BACKBOXUSERPASS #enter your user Password for BackBox
externalApiURL = BackBoxBaseURL + '/rest/data/api/'
internalApiURL = BackBoxBaseURL + '/rest/data/'

#custom information about files and jobs we will use
devices_to_upgrade_file = './Devices_to_add.csv' #Path to a CSV with just external IDs in the first column


### Here we will import out list of Device IDs from a CSV. These could come from anywhere in reality, this could be from your Device Database API or just from a CSV as in this example. Key here is that we get some sort of datastructure that will help us identify devices we want to operate on.

In [ ]:
key = ''
devicesToAdd = []
with open(devices_to_upgrade_file, mode='r', newline='', encoding='utf-8-sig') as csvfile:
    reader = csv.reader(csvfile, delimiter = ' ', quotechar='|')
    first = True
    for item in reader:
        fields = item[0].split(",")
        if first:
            keys = fields
            first = False
        else:
            devicesToAdd += [dict(zip(keys,fields))]

print(devicesToAdd)


In [ ]:
session = requests.Session()

### Let's go get a connection to our backbox server. One way to authenticate here is to get our cookies and use them to maintain our session

In [ ]:
#reach out to BackBox and get the login page
response = session.get(BackBoxBaseURL, verify=False)

### Now that we have our session cookies we can send username and password to login

In [ ]:
headers = {
    'Content-Type': 'application/x-www-form-urlencoded; charset=utf-8',
}

data = {
    'j_username': BackBoxUser,
    'j_password': BackBoxPassword,
}

response = session.post(BackBoxBaseURL+'/j_security_check', cookies=response.cookies, headers=headers, data=data, verify=False)

### After logging in adding a device requires Name, IP, and (Vendor, Product, Version, Option) ids. An external ID helps us sync with systems outside BackBox. We had all these in our CSV file.
### We can get these Vendor, Product, Version and Option ID's by searching at https://support.backbox.com/s/device-lookup 

In [ ]:
for device in devicesToAdd:
    headers ={'Content-Type': 'application/json', "Accept": "*/*"}
    data ={
        "userName":device['USER_NAME'],
        "password":device['PASSWORD'],
        "deviceName": device['DEVICE_NAME'],
        "serverIP": device['IP_ADDRESS'],
        "vendorId":device['VENDOR_ID'],
        "productId":device['PRODUCT_ID'],
        "versionId":device['VERSION_ID'],
        "optionId": device['OPTION_ID'],
        "externalId": device['EXTERNAL_ID']
    }
    deviceAdded = session.post(externalApiURL +'device', cookies=response.cookies, headers=headers, json=data, verify=False)
    if deviceAdded.ok:
        print(deviceAdded.json())
    else:
        print(deviceAdded.reason)

In [ ]:
deviceAdded.text

In [ ]:
session.close()